In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation, NMF

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)


In [4]:
ads = pd.read_csv('amazon_ten_topic_data.csv', names=['ad'])
ads.head()

,ad
0,JR2 Shining Double Row Crystal Metal License P...
1,"[Upgraded]BOYISEN Trailer Hitch Pin Lock, 1/2""..."
2,SUMEX 5070138b Hubcap
3,Rugged Ridge 15201.16 1.25 Wheel Spacer for 20...
4,"Bondhus 33034 Set of 8 Star T-handles, Graduat..."


In [42]:
ads = ads.sample(frac=1)
ads.dropna(inplace=True)
ads.describe()

,ad
count,10000
unique,9934
top,USB Printer with 3 Year Printer Warranty
freq,6


In [43]:
ads.head()

,ad
7108,SIWY Women's Gaby High-Waisted Skinny Jeans in...
9751,Akribos Swarovski Crystal Women's Watch - Diam...
2269,Epson PictureMate Express Printer
226,Extreme Max 5001.5754 3-Position Motorcycle Wh...
211,OEM Suzuki JR 50 Junior Front or Rear Wheel Sp...


In [44]:
ads_unique = pd.DataFrame(ads['ad'].unique(), columns = ['ad'])
ads_unique.describe()

,ad
count,9934
unique,9934
top,"Moen BP6908CH Madison Paper Holder, Chrome"
freq,1


### LDA

In [45]:
cv = CountVectorizer(max_df=0.9, min_df=2, stop_words='english')

# document term matrix
dtm = cv.fit_transform(ads_unique['ad'])

dtm

<9934x6579 sparse matrix of type '<class 'numpy.int64'>'
	with 105479 stored elements in Compressed Sparse Row format>

In [46]:
LDA = LatentDirichletAllocation(n_components=10,random_state=42)

# This can take awhile, we're dealing with a large amount of documents!
LDA.fit(dtm)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=10, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [47]:
ads_unique.head()

,ad
0,SIWY Women's Gaby High-Waisted Skinny Jeans in...
1,Akribos Swarovski Crystal Women's Watch - Diam...
2,Epson PictureMate Express Printer
3,Extreme Max 5001.5754 3-Position Motorcycle Wh...
4,OEM Suzuki JR 50 Junior Front or Rear Wheel Sp...


In [48]:
import random

for i in range(10):
    random_word_id = random.randint(0,6579)
    print(cv.get_feature_names()[random_word_id])

dpi
plain
authority
classico
buttons
ipad
trans
traveler
lightning
protective


In [49]:
len(LDA.components_)

10

In [50]:
LDA.components_.shape

(10, 6579)

In [51]:
for index,topic in enumerate(LDA.components_):
    print(f'THE TOP 10 WORDS FOR TOPIC #{index}')
    print([cv.get_feature_names()[i] for i in topic.argsort()[-15:]])
    print('\n')

THE TOP 10 WORDS FOR TOPIC #0
['hub', 'printer', 'center', 'cover', 'cap', 'stem', 'pack', 'thermal', 'usb', 'black', 'car', 'caps', 'valve', 'tire', 'wheel']


THE TOP 10 WORDS FOR TOPIC #1
['brown', 'puma', 'home', 'stool', 'shoes', 'fashion', 'storage', 'trainers', 'black', 'white', 'chair', 'sneakers', 'low', 'sneaker', 'women']


THE TOP 10 WORDS FOR TOPIC #2
['dell', 'ips', 'raspberry', 'touch', 'pi', 'hd', 'vga', 'black', 'hdmi', 'display', 'screen', 'led', 'lcd', 'inch', 'monitor']


THE TOP 10 WORDS FOR TOPIC #3
['gauge', 'portable', 'digital', 'scanner', 'head', 'canon', 'wireless', 'car', 'hp', 'color', 'air', 'pressure', 'tire', 'photo', 'printer']


THE TOP 10 WORDS FOR TOPIC #4
['wired', 'cancelling', 'iphone', 'microphone', 'headset', 'black', 'stereo', 'mic', 'earphones', 'noise', 'earbuds', 'wireless', 'bluetooth', 'ear', 'headphones']


THE TOP 10 WORDS FOR TOPIC #5
['leather', 'casual', 'new', 'fashion', 'sneaker', 'daypack', 'girls', 'men', 'black', 'laptop', 'trave

In [52]:
topic_results = LDA.transform(dtm)

In [53]:
topic_results.shape

(9934, 10)

In [56]:
ads_unique['Topic'] = topic_results.argmax(axis=1)

my_topic_dictionary = {0: 'Tire & Wheels', 
                       1: 'Sneakers', 
                       2: 'Furniture', 
                       3: 'Printer', 
                       4: 'Headphones', 
                       5: 'Backpack', 
                       6: 'Watches',
                       7: 'Jeans', 
                       8: 'bath', 
                       9: 'Monitor'}

ads_unique['Product Type'] = ads_unique['Topic'].map(my_topic_dictionary)

ads_unique

,ad,Topic,Product Type
0,SIWY Women's Gaby High-Waisted Skinny Jeans in...,6,Watches
1,Akribos Swarovski Crystal Women's Watch - Diam...,6,Watches
2,Epson PictureMate Express Printer,3,Printer
3,Extreme Max 5001.5754 3-Position Motorcycle Wh...,0,Tire & Wheels
4,OEM Suzuki JR 50 Junior Front or Rear Wheel Sp...,0,Tire & Wheels
...,...,...,...
9929,Magna Cart Ideal 150 lb Capacity Steel Folding...,3,Printer
9930,Dr.Scholls Women's Kinney Lace Fashion Sneaker...,1,Sneakers
9931,Miss Me Women's Hailey Metallic Lace Embellish...,6,Watches
9932,"MOZIONI School Backpack, Star #1 Collection, L...",5,Backpack


In [58]:
a = [1,2,4,5, 10, 16]
b = [1,3,5,6]
sim = set(a).intersection(b)
print(sim)

set(a) - sim

# MAKE INTERCECTION LAATER

{1, 5}


{2, 4, 10, 16}

### NMF

In [66]:
tfidf = TfidfVectorizer(max_df=0.95, min_df=2, stop_words='english')

dtm = tfidf.fit_transform(ads_unique['ad'])

dtm

<9934x6579 sparse matrix of type '<class 'numpy.float64'>'
	with 105479 stored elements in Compressed Sparse Row format>

In [67]:
nmf_model = NMF(n_components=10,random_state=42)

# This can take awhile, we're dealing with a large amount of documents!
nmf_model.fit(dtm)

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
    n_components=10, random_state=42, shuffle=False, solver='cd', tol=0.0001,
    verbose=0)

In [68]:
len(tfidf.get_feature_names())

6579

In [69]:
for index,topic in enumerate(nmf_model.components_):
    print(f'THE TOP 15 WORDS FOR TOPIC #{index}')
    print([tfidf.get_feature_names()[i] for i in topic.argsort()[-15:]])
    print('\n')

THE TOP 15 WORDS FOR TOPIC #0
['silver', 'ladies', 'dial', 'womens', 'gold', 'analog', 'women', 'analogue', 'model', 'leather', 'stainless', 'steel', 'strap', 'quartz', 'watch']


THE TOP 15 WORDS FOR TOPIC #1
['wired', 'sports', 'black', 'microphone', 'headset', 'cancelling', 'stereo', 'mic', 'earphones', 'noise', 'earbuds', 'wireless', 'ear', 'bluetooth', 'headphones']


THE TOP 15 WORDS FOR TOPIC #2
['straight', 'size', 'womens', 'waist', 'mid', 'stretch', 'ankle', 'pants', 'jean', 'high', 'denim', 'rise', 'women', 'skinny', 'jeans']


THE TOP 15 WORDS FOR TOPIC #3
['lg', '24', 'vga', 'dell', 'display', 'lit', '27', 'hd', 'ips', 'hdmi', 'screen', 'led', 'lcd', 'inch', 'monitor']


THE TOP 15 WORDS FOR TOPIC #4
['laserjet', 'scanner', 'pixma', 'copier', 'printing', 'monochrome', 'lexmark', 'inkjet', 'wireless', 'laser', 'color', 'canon', 'photo', 'hp', 'printer']


THE TOP 15 WORDS FOR TOPIC #5
['casual', 'charging', 'business', 'bookbag', '15', 'black', 'college', 'daypack', 'girls'

In [70]:
topic_results = nmf_model.transform(dtm)

ads_unique.head()

,ad,Topic,Product Type,NMF Topic,NMF Topic Label
0,SIWY Women's Gaby High-Waisted Skinny Jeans in...,6,Watches,2,Jeans
1,Akribos Swarovski Crystal Women's Watch - Diam...,6,Watches,0,Watches
2,Epson PictureMate Express Printer,3,Printer,4,Printer
3,Extreme Max 5001.5754 3-Position Motorcycle Wh...,0,Tire & Wheels,9,Tires & Wheels
4,OEM Suzuki JR 50 Junior Front or Rear Wheel Sp...,0,Tire & Wheels,9,Tires & Wheels


In [65]:
ads_unique['NMF Topic'] = topic_results.argmax(axis=1)

my_topic_dictionary = {0: 'Watches', 
                       1: 'Headphones', 
                       2: 'Jeans', 
                       3: 'Monitor', 
                       4: 'Printer', 
                       5: 'Backpack', 
                       6: 'SNEAKER',
                       7: 'Bath', 
                       8: 'Furniture', 
                       9: 'Tires & Wheels'}

ads_unique['NMF Topic Label'] = ads_unique['NMF Topic'].map(my_topic_dictionary)
ads_unique

,ad,Topic,Product Type,NMF Topic,NMF Topic Label
0,SIWY Women's Gaby High-Waisted Skinny Jeans in...,6,Watches,2,Jeans
1,Akribos Swarovski Crystal Women's Watch - Diam...,6,Watches,0,Watches
2,Epson PictureMate Express Printer,3,Printer,4,Printer
3,Extreme Max 5001.5754 3-Position Motorcycle Wh...,0,Tire & Wheels,9,Tires & Wheels
4,OEM Suzuki JR 50 Junior Front or Rear Wheel Sp...,0,Tire & Wheels,9,Tires & Wheels
...,...,...,...,...,...
9929,Magna Cart Ideal 150 lb Capacity Steel Folding...,3,Printer,7,Bath
9930,Dr.Scholls Women's Kinney Lace Fashion Sneaker...,1,Sneakers,6,SNEAKER
9931,Miss Me Women's Hailey Metallic Lace Embellish...,6,Watches,2,Jeans
9932,"MOZIONI School Backpack, Star #1 Collection, L...",5,Backpack,5,Backpack
